<a href="https://colab.research.google.com/github/myidispg/PytorchProjects/blob/master/Private%20AI/Differential_Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch

In [4]:
db = torch.randn(5000) > 0.5
db

tensor([1, 0, 1,  ..., 0, 0, 1], dtype=torch.uint8)

In [28]:
def generate_pdb(db, index):
  
  return torch.cat((db[:index], db[index+1:]))



def get_parallel_dbs(db):
  
  pdbs = torch.zeros((len(db), len(db)-1))
  
  for i in range(len(db)):
    pdb = generate_pdb(db, i)
    pdbs[i] = pdb
    
  return pdbs
   
def get_db_pdbs():
  
  db = torch.randn(5000) > 0.5
  
  pdbs = get_parallel_dbs(db)
  
  return db, pdbs

db, pdbs = get_db_pdbs()
print(db.shape)
print(pdbs.shape)

torch.Size([5000])
torch.Size([5000, 4999])
